# Projeto final - App de receitas
Digite um ou mais ingredientes e encontre receitas que podem ser feitas com esses ingredientes

## 1. Importando bibliotecas

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import numpy as np
import re
from time import sleep
import random
from unidecode import unidecode

## 2 WebScrapping

#### Site 1 para banco de dados - https://www.daninoce.com.br/

In [ ]:
site='https://www.daninoce.com.br/receitas/'
rq=requests.get(site)

## 3.1 Definindo categorias

In [ ]:
soup=BeautifulSoup(rq.content)

In [ ]:
soup.find_all('a')

In [ ]:
categorias=[item['href'] if 'categoria-de-receita' in item['href'] else 0 for item in soup.find_all('a',attrs={'class':'mark'})]

In [ ]:
categorias.remove(0)

In [ ]:
len(categorias)

In [ ]:
categorias[0].strip('/').split('https://www.daninoce.com.br/categoria-de-receita/')[1]

In [ ]:
categorias.remove(0)
len(categorias)

In [ ]:
categorias_nome=[item.strip('/').split('https://www.daninoce.com.br/categoria-de-receita/')[1] for item in categorias]

In [ ]:
categorias=list(set(categorias))

In [ ]:
len(categorias)

## 3.2 Defininfo receitas

In [ ]:
lista1=[]
for item in categorias:
    rq2=requests.get(item)
    soup2=BeautifulSoup(rq2.content)
    receitas=[item['href'] if '/receitas/' in item['href'] else 0 for item in soup2.find_all('a')]
    receitas=list(set(receitas))
    lista1.append(receitas)

In [ ]:
#Removendo os zeros das listas
for item in lista1:
    item.remove(0)

In [ ]:
lista1

In [ ]:
categorias_dict={}
for item in range(len(categorias_nome)):
    categorias_dict.update({categorias_nome[item]:lista1[item]})
print(categorias_dict)

In [ ]:
if 'https://www.daninoce.com.br/receitas/massa-para-bolo-de-chocolate-ou-petit-gateau/' in categorias_dict:
    print(categorias_dict['https://www.daninoce.com.br/receitas/massa-para-bolo-de-chocolate-ou-petit-gateau/'])

In [ ]:
categorias_dict['whoopie-pie']

In [ ]:
categorias_dict['whoopie-pie']

In [ ]:
def buscar_dict (x):
    
    '''Função pra extraix os valores (que são as categorias) do dicionario para criar uma coluna com esses valores.'''
    
    for item in categorias_nome:
        lista_dict=categorias_dict[item]
        if x in lista_dict:
            return item

In [ ]:
link='https://www.daninoce.com.br/receitas/whoopie-pie-de-floresta-negra/'

In [ ]:
buscar_dict(link)

### 3.3 Criando DataFrame + coluna nome e coluna sites

In [ ]:
from functools import reduce

In [ ]:
def my_sum(a,b):
    
    '''Função pra concatenar listas.'''
    
    return a + b

In [ ]:
# checando all_receipes

In [ ]:
all_receipes=reduce(my_sum,lista1)

In [ ]:
all_recipes=all_receipes

In [ ]:
len(all_recipes)

In [ ]:
all_recipes[0:3]

In [ ]:
'https://www.daninoce.com.br/receitas/trufa-vegana-no-copinho-funcional/'.split('https://www.daninoce.com.br/receitas/')[1]

In [ ]:
name=[item.strip('/').split('https://www.daninoce.com.br/receitas/')[1] for item in all_receipes]

In [ ]:
df=pd.DataFrame(name,columns=['nome'])

In [ ]:
df['sites']=all_recipes

## 3.3 Definindo Ingredientes

In [ ]:
# testando ants de aplicar no codigo - soup3.find_all('li',attrs={'itemprop':'recipeIngredient'})[0].text

In [ ]:
lista2=[]
for item in tqdm(all_receipes):
    rq3=requests.get(item)
    soup3=BeautifulSoup(rq3.content)
    ingredientes=[item.text for item in soup3.find_all('li',attrs={'itemprop':'recipeIngredient'})]
    ingredientes=list(set(ingredientes))
    lista2.append(ingredientes)
df['ingredientes']=lista2

In [ ]:
df['ingredientes'].head(3)

In [ ]:
# usar em outro momento
#df['ingredientes']=df['ingredientes'].apply(lambda x:eval(x))

In [ ]:
','.join(df['ingredientes'][0])

In [ ]:
df['ingredientes']=[', '.join(item) for item in df['ingredientes']]

In [ ]:
df['ingredientes'][0].lower()

In [ ]:
df['ingredientes']=[item.lower() for item in df['ingredientes']]

## 3.4 Definindo modo de preparo

In [ ]:
soup3.find_all('li',attrs={'itemprop':'recipeInstructions'})[0].text

In [ ]:
lista3=[]
for item in tqdm(all_receipes):
    rq4=requests.get(item)
    soup4=BeautifulSoup(rq4.content)
    modo_de_preparo=[item.text for item in soup4.find_all('li',attrs={'itemprop':'recipeInstructions'})]
    modo_de_preparo=list(set(modo_de_preparo))
    lista3.append(modo_de_preparo)
df['modo de preparo']=lista3

In [ ]:
df.head(3)

## 3.5 Definindo orientações sobre a receita

In [ ]:
soup

In [ ]:
# só checando se o loop vai rolar - soup5.find_all('p',attr={'class':"text text-quote quote-recipe text-medium text-upper"})

In [ ]:
tempo_de_espera=[]
tempo_de_preparo=[]
rendimento=[]
tempo_de_cozimento=[]
error=[]
for item in tqdm(all_receipes):
    
    try:
        rq5=requests.get(item)
        soup5=BeautifulSoup(rq5.content)

        orientaçoes_preparo=[item.text for item in soup5.find_all('span', attrs={'class':"prepare-time"})]
        orientaçoes_preparo=list(set(orientaçoes_preparo))
        tempo_de_preparo.append(orientaçoes_preparo)

        orientaçoes_cozimento=[item.text for item in soup5.find_all('span', attrs={'class':"cook-time"})]
        orientaçoes_cozimento=list(set(orientaçoes_cozimento))
        tempo_de_cozimento.append(orientaçoes_cozimento)

        orientaçoes_espera=[item.text for item in soup5.find_all('span', attrs={'class':"wait-time"})]
        orientaçoes_espera=list(set(orientaçoes_espera))
        tempo_de_espera.append(orientaçoes_espera)

        orientaçoes_rendimento=[item.text for item in soup5.find_all('span', attrs={'class':"rendiment"})]
        orientaçoes_rendimento=list(set(orientaçoes_rendimento))
        rendimento.append(orientaçoes_rendimento)

    except:
        error.append(item)
        
df['tempo_de_preparo']=tempo_de_preparo
df['tempo_de_cozimento']=tempo_de_cozimento
df['tempo_de_espera']=tempo_de_espera
df['rendimento']=rendimento

### Criando as colunas Categorias

In [ ]:
df['categorias']=df['sites'].apply(lambda x: buscar_dict(x))

In [ ]:
df.shape

## 4. Tratando as colunas

### 4.1 Tempo de preparo

In [ ]:
df['tempo_de_preparo'][0]

In [ ]:
tdp=[item[0].strip() for item in df['tempo_de_preparo']]

In [ ]:
df['tempo_de_preparo']

In [ ]:
def h_min (x):
    
    '''Função pra converter tempo de horas pra min (caso o tempo esteja em horas.)
       Se o tempo ja estiver em minutos, somente retornar o tempo.'''
    
    if 'horas' in x:
        regex=re.findall('\d+',x)
        if len(regex)==1:
            tempo=int(regex[0])
            return str(tempo*60)+' minutos'
        elif len(regex)==2:
            tempo=int(regex[1])
            return str(tempo*60)+' minutos'
    else:
        return x

In [ ]:
df['tempo_de_preparo']=df['tempo_de_preparo'].apply(lambda x: h_min(x[0]))

In [ ]:
pattern='\d*'
df['tempo_de_preparo_min']=[re.findall(pattern,item)[0] for item in tdp]

In [ ]:
df['tempo_de_preparo_min']

In [ ]:
df=df.drop(columns=['tempo_de_preparo'])

### 4.2 Tempo de cozimento

In [ ]:
df['tempo_de_cozimento']

In [ ]:
df['tempo_de_cozimento']=df['tempo_de_cozimento'].apply(lambda x: h_min(x[0]))

In [ ]:
#df['tempo_de_cozimento']=df['tempo_de_cozimento'].apply(lambda x:eval(x))

In [ ]:
#definir tdc
tdc=[item.strip() for item in df['tempo_de_cozimento']]

In [ ]:
pattern='\d*'
df['tempo_de_cozimento_min']=[re.findall(pattern,item)[0] for item in tdc]

In [ ]:
df['tempo_de_cozimento_min']

In [ ]:
df=df.drop(columns=['tempo_de_cozimento'])

## 4.4 Tempo de espera

In [ ]:
df['tempo_de_espera'].head()

In [ ]:
df['tempo_de_espera']=df['tempo_de_espera'].apply(lambda x: h_min(x[0]))

In [ ]:
df['tempo_de_espera'].head()

In [ ]:
#df['tempo_de_espera'].apply(lambda x:eval(x))

In [ ]:
tde = [item.strip() for item in df['tempo_de_espera']]

In [ ]:
tde

In [ ]:
pattern='\d*'
df['tempo_de_espera_min']=[re.findall(pattern,item)[0] for item in tde]

In [ ]:
df=df.drop(columns=['tempo_de_espera'])

In [ ]:
df.head(12)

## 4.5 Rendimento

In [ ]:
df['rendimento'][0]

In [ ]:
df['rendimento']=[item[0] for item in df['rendimento']]

In [ ]:
'''lissta=[]
for item in df['rendimento']:
    if item in lissta:
        pass
    else:
        lissta.append(item)
print(lissta)'''

In [ ]:
'''rnp=[item[0].strip() for item in df['rendimento']]'''

In [ ]:
'''pattern='\d*'
[re.findall(pattern,item)[0] for item in rnp]'''

In [ ]:
'''df['rendimento_numero_pessoas']=rnp'''

Obs. Percebi que rendimento nem sempre é para o numero de porções, mas também tem numeros de unidades, ou total de peso.
Por conta disso, optei por não alterar esse dado dessa coluna.

## 5. Salvando df no pc para agilizar processos durante o projeto

In [ ]:
df.to_csv('df_Projeto_Final_V1.csv')

In [2]:
df=pd.read_csv('./df_Projeto_Final_v1.csv')

In [ ]:
df.head(2)

### 5.1 Tirando os acentos de todo o Data Frame

In [ ]:
#! pip install unidecode

In [ ]:
df=df.applymap(lambda x: unidecode(x) if type(x) == str else x)

## Criando indices para as receitas

In [ ]:
df.head()

In [ ]:
df=df.rename(columns={'Unnamed: 0':'indice'})

## Colocando o código pra funcionar

In [9]:
count = 0
index = []
variavel_2 = 1
busca=0
ingrediente_1=1
print('\n','´  ´  '*20)
print(('~ - '*8),'Seja vem vindo ao site de receitas do Victor Jackman! ',('~ - '*8),'\n')
sleep(2)
print(('- _ '*6),'Espero que encontre ótimas receitas, e que se divirta muito no processo!',('- _ '*6),'\n')
sleep(2)
print(('- _ '*4),'Não esquece de me dizer como ficou a receita, basta postar no insta usando @vijackman',('- _ '*4),'\n')
sleep(2)
print('Antes de começar vale lembrar que esse site utiliza receitas do site da Dani Noce, que é uma confeiteira profissional que eu admiro muito!','\n')
sleep(2)
print('´  ´  '*20,'\n')
while variavel_2 > 0:
    if busca != 2:
        busca=int(input('Você quer buscar a receita pelo nome da receita [ 1 ] ou por ingrediente [ 2 ] ? '))
    if busca == 2:
        print('.')
        if ingrediente_1 == 1:
            ingrediente_1=str(input('Digite o Ingrediente (letras minusculas e sem acentos):'))
        
            for i in range(len(df['ingredientes'])):
                if ingrediente_1 in df['ingredientes'][i]:
                    count += 1
                    index.append(i)
                    print('\n','Receita ',df.loc[i,'indice'],': _-_-_-_-_-_-_-_-_-_-_-_','Receita:',df['nome'][i],'\n\n')

            if ingrediente_1 not in df['ingredientes'][i] and count ==0:
                    print('Não localizei nenhuma receita com esse ingrediente, tente novamente: ')
                    ingrediente_1=str(input('Digite o Ingrediente (letras minusculas e sem acentos): '))

            print('\n','- - - - - - - - - - - - - - - - - - - - - -')        
            print('Foram encontradas ',count,' receitas!')
            print('- - - - - - - - - - - - - - - - - - - - - -')
            variavel=int(input('Selecione uma receita (numero do indice): '))
            print('- - - - - - - - - - - - - - - - - - - - - -') 


            if variavel == df['indice'][variavel]:
                print('\n\n','-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- Receita ',df.loc[variavel,'nome'],': -_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_','\n')
        
                print('Categoria: ',df['categorias'][variavel],'\n')  
                print('Tempo de preparo:',df['tempo_de_preparo_min'][variavel],'minutos')
                print('Tempo de espera:',df['tempo_de_espera_min'][variavel],'minutos')
                print('Tempo de cozimento:',df['tempo_de_cozimento_min'][variavel],'minutos')
                print('Rendimento:',df['rendimento'][variavel],'\n')

                print('Ingredientes','\n',df['ingredientes'][variavel],'\n')
                print('Modo de preparo','\n',df['modo de preparo'][variavel],'\n\n')

                sleep(5)

                mask=((df['categorias']==df['categorias'][variavel]) & (df['nome'] != df['nome'][variavel]))

                print('\n','Outras receitas que você também pode se interessar: ','\n')

                passo_1 = df[mask]['nome'] 
                passo_2 = passo_1.sample(3)
                sugestoes = list(passo_2)

                lista_recomendada = list(passo_2.index)

                print(lista_recomendada[0],'- _  - _  - _',sugestoes[0],'\n')
                print(lista_recomendada[1],'- _  - _  - _',sugestoes[1],'\n')
                print(lista_recomendada[2],'- _  - _  - _',sugestoes[2],'\n')

                variavel_2 = int(input('Selecione uma receita (numero do indice) ou 0 para encerrar: '))

                if variavel_2 == df['indice'][variavel_2]:

                    print('\n\n','-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- Receita ',df.loc[variavel_2,'nome'],': -_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_','\n')

                    print('\n','Categoria: ',df['categorias'][variavel_2],'\n')  
                    print('Tempo de preparo:',df['tempo_de_preparo_min'][variavel_2],'minutos')
                    print('Tempo de espera:',df['tempo_de_espera_min'][variavel_2],'minutos')
                    print('Tempo de cozimento:',df['tempo_de_cozimento_min'][variavel_2],'minutos')
                    print('Rendimento:',df['rendimento'][variavel_2],'\n')

                    print('Ingredientes','\n',df['ingredientes'][variavel_2],'\n')
                    print('Modo de preparo','\n',df['modo de preparo'][variavel_2],'\n\n')

                    sleep(5)

                    mask=((df['categorias']==df['categorias'][variavel_2]) & (df['nome'] != df['nome'][variavel_2]))

                    print('\n','Outras receitas que você também pode se interessar: ','\n')

                    passo_1 = df[mask]['nome']
                    passo_2 = passo_1.sample(3)
                    sugestoes = list(passo_2)

                    lista_recomendada = list(passo_2.index)

                    print(lista_recomendada[0],'- _  - _  - _',sugestoes[0],'\n')
                    print(lista_recomendada[1],'- _  - _  - _',sugestoes[1],'\n')
                    print(lista_recomendada[2],'- _  - _  - _',sugestoes[2],'\n')

                    variavel_2 = int(input('Selecione uma receita (numero do indice) ou 0 para encerrar: '))

            else:
                variavel_2=int(input('Não localizei esse ingrediente. Tente novamente : '))

    elif busca == 1:
        nome_da_receita=str(input('Digite o nome da receita (letras minusculas e sem acento): '))
        for n in range(len(df['ingredientes'])):
            if nome_da_receita in df['nome'][n]:
                count += 1
                index.append(n)
                print(' Receita ',df.loc[n,'indice'],': _-_-_-_-_-_-_-_-_-_-_-_','Receita:',df['nome'][n],'\n\n')

        if nome_da_receita not in df['nome'][n] and count == 0:
            print('Não localizei nenhuma receita com esse nome, tente novamente: ')
            nome_da_receita = str(input('Digite o nome da receita (letras minusculas e sem acentos):'))

        print('- - - - - - - - - - - - - - - - - - - - - -')        
        print('Foram encontradas ',count,' receitas!')
        print('- - - - - - - - - - - - - - - - - - - - - -')
        variavel_2 = int(input('Selecione uma receita (numero do indice): '))
        print('- - - - - - - - - - - - - - - - - - - - - -')         
        
        if variavel_2 == df['indice'][variavel_2]:

            print('Categoria: ',df['categorias'][variavel_2])  
            print('Tempo de preparo:',df['tempo_de_preparo_min'][variavel_2],'minutos')
            print('Tempo de espera:',df['tempo_de_espera_min'][variavel_2],'minutos')
            print('Tempo de cozimento:',df['tempo_de_cozimento_min'][variavel_2],'minutos')
            print('Rendimento:',df['rendimento'][variavel_2],'\n')

            print('Ingredientes','\n',df['ingredientes'][variavel_2],'\n')
            print('Modo de preparo','\n',df['modo de preparo'][variavel_2],'\n')

            sleep(5)

            mask=((df['categorias']==df['categorias'][variavel_2]) & (df['nome'] != df['nome'][variavel_2]))

            print('\n',('~ - '*8),'Outras receitas que você também pode se interessar: ',('~ - '*8),'\n')

            passo_1 = df[mask]['nome']
            passo_2 = passo_1.sample(3)
            sugestoes = list(passo_2)

            lista_recomendada = list(passo_2.index)

            print(lista_recomendada[0],'- _  - _  - _',sugestoes[0],'\n')
            print(lista_recomendada[1],'- _  - _  - _',sugestoes[1],'\n')
            print(lista_recomendada[2],'- _  - _  - _',sugestoes[2],'\n')

            variavel_2 = int(input('Selecione uma receita (numero do indice) ou 0 para encerrar: '))
        
        else:
            variavel_2 = int(input('Não tem essa opção, tente novamente (digite pelo numero do indice): '))
            
    else:
        busca=int(input('Não existe essa opção, tente novamente: nome da receita [ 1 ] ou por ingrediente [ 2 ] '))

print('\n\n','~ - '*30)
print((' '*30),'Obrigado por usar o meu site lindo de receitas! :p ')
print('~ - '*30,'\n\n')


 ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  
~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ -  Seja vem vindo ao site de receitas do Victor Jackman!  ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ -  

- _ - _ - _ - _ - _ - _  Espero que encontre ótimas receitas, e que se divirta muito no processo! - _ - _ - _ - _ - _ - _  

- _ - _ - _ - _  Não esquece de me dizer como ficou a receita, basta postar no insta usando @vijackman - _ - _ - _ - _  

Antes de começar vale lembrar que esse site utiliza receitas do site da Dani Noce, que é uma confeiteira profissional que eu admiro muito! 

´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´  ´   

Você quer buscar a receita pelo nome da receita [ 1 ] ou por ingrediente [ 2 ] ? 2
.
Digite o Ingrediente (letras minusculas e sem acentos):cacau

 Receita  0 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: nougat-de-chocolate-do-chef-christophe-rhedon 




 Receita  217 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: suspiro-de-chocolate-com-creme-de-chocolate 



 Receita  221 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: brownie-de-nozes 



 Receita  222 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: sanduiche-de-sorvete-de-chocolate-e-amora 



 Receita  225 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: barrinha-de-chocolate-com-coco-low-carb 



 Receita  228 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: tiramisu-cointreau 



 Receita  234 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: buque-de-cupcake 



 Receita  235 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: petit-gateau-low-carb 



 Receita  238 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: tiramisu-low-carb 



 Receita  239 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: brownie-low-carb 



 Receita  240 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: smore-de-forno 



 Receita  243 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: rabanada-de-chocolate 



 Receita  245 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: brownie-fit-com-batata-doce-sem-glutenlactose 



 Receita  247 : _-_-_-_-_-_-_-_-_-_-_-_ Recei

 Receita  1078 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: cupcakes-de-oreo 



 Receita  1081 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: cupcake-triple-chocolate 



 Receita  1082 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: cupcakes-prestigio 



 Receita  1084 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: cupcakes-de-chocolate-com-cream-cheese-de-laranja 



 Receita  1086 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: cupcake-red-velvet 



 Receita  1090 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: cupcakes-hostess 



 Receita  1096 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: cupcake-de-iogurte-com-chocolate 



 Receita  1107 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: massa-para-bolo-de-chocolate-ou-petit-gateau 



 Receita  1109 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: pate-sucree-de-chocolate-amargo 



 Receita  1115 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: massa-de-bolo-de-iogurte-com-chocolate 



 Receita  1116 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: massa-para-bolo-red-velvet 



 Receita  1122 : _-_-_-_-_-_-_-_-_-_-_-_ Receita: massa-para-bolo-double-chocolate-


 Outras receitas que você também pode se interessar:  

4 - _  - _  - _ pirulito-de-marshmallow-de-framboesa 

1 - _  - _  - _ marshmallow-caseiro 

3 - _  - _  - _ marshmallow-de-baunilha-e-menta 

Selecione uma receita (numero do indice) ou 0 para encerrar: 0


 ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - 
                               Obrigado por usar o meu site lindo de receitas! :p 
~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ - ~ -  




In [ ]:
df['nome'][0]

In [ ]:
if 'chocolate' in df['nome'][0]:
    print(1)

## Criar um sistema de recomendação

In [ ]:
# colocado no codigo acima! 

'''mask=((df['categorias']==df['categorias'][variavel]) & (df['nome'] != df['nome'][variavel]))

    print('\n\n','Outras receitas que você também pode se interessar: ','\n')

    passo_1 = df[mask]['nome']
    passo_2 = passo_1.sample(3)
    passo_3 = list(passo_2)
    

    print('Receita 1: ','\n')

    print('Receita 2: ','\n')

print('Receita 3: ','\n')'''

## Deploy

In [ ]:
#! pip install streamlit

In [ ]:
# Feito nos arquivos no pc pra transferir a funcionalidade do codigo para uma pagina

## Coisas que ficaram pra ser resolvidas

1. Categoria gelatina só tem 3 receitas na categoria, por isso, quando ela vai sugerir, elasó tem 2 pra fazer sugestão. 
Como eu peço 3 sugestões,acaba dando erro. 

2. depois que coloco o numero da sugestão no variavel_2 ele tras a mesma receita que solicitei.todos os dados (categoria, orientações, ingredientes e modo de preparo)

3. Da primeira vez que mostra as sugestões (seguindo o caminho 2, caso eu digite 0, ele não encerra o app, mas na segunda vez q ele mostra sugestões, ai funciona normal.)

## Coisas que posso agregar no projeto

1. Agregar novos sites de receitas no df pra ter um banco de dados maior

2. agregar um sistema que faz o calculo de valor de cada item usado nas receitas, e incluir o custo total aproximado da receita no código.

3. Colocar 3 ingredientes ao invés de 1 só

4. Enfeitar mais o Deploy